In [6]:
import pandas as pd
import numpy as np
import os

from scipy.signal import convolve2d as conv2d
import time
import cv2
import nibabel as nb
# from imea.measure_2d.micro import fractal_dimension_boxcounting as fract_dim

import warnings
warnings.filterwarnings('ignore')

In [7]:
def lac1(x):
    bin_ = np.zeros((x.shape[0], x.shape[1]))
    bin_[x!=0] = 1
    bin_ = bin_.astype(np.uint8)
    
    contours = cv2.findContours(image = bin_, mode = cv2.RETR_TREE, method = cv2.CHAIN_APPROX_SIMPLE)[0]
    contours = sorted(contours, key = cv2.contourArea, reverse = True)
    
    if len(contours) == 0:
        return 0
    
    c_0 = contours[0]               #Get the 4 points of the bounding rectangle
    xx, yy, w, h = cv2.boundingRect(c_0)
    X  =  x[yy:yy+h, xx:xx+w]

    box_size =  [i for i in range(2, 240) if ((i)**2 < (0.45)*(X.shape[0]* X.shape[1])) and i <= min(X.shape[0], X.shape[1])]

    LAMBDA = []
    
    XX = np.zeros((X.shape[0], X.shape[1]))
    
    XX[X==1] = 1
  
    for box in box_size:
        count = []
        count, edge= np.histogram(np.ravel(conv2d(XX, np.ones((box, box)), mode = 'valid')), bins = [i for i in range(0, (box**2) + 2)])#(box**2) + 1)
        q = count/(XX.shape[0] - box + 1)**2
        s = np.array([i for i in range(0,box**2 + 1)])
        lam_bda =  sum((s**2)*q)/(sum(q*s))**2
        
     
        LAMBDA.append(lam_bda)
        
    return np.nanmean(LAMBDA)

In [8]:
def lac2(x):
    bin_ = np.zeros((x.shape[0], x.shape[1]))
    bin_[x!=0] = 1
    bin_ = bin_.astype(np.uint8)

    contours = cv2.findContours(image = bin_, mode = cv2.RETR_TREE, method = cv2.CHAIN_APPROX_SIMPLE)[0]
    contours = sorted(contours, key = cv2.contourArea, reverse = True)
    
    if len(contours) == 0:
        return 0
    
    c_0 = contours[0]               #Get the 4 points of the bounding rectangle
    xx, yy, w, h = cv2.boundingRect(c_0)
    X  =  x[yy:yy+h, xx:xx+w]

    box_size =  [i for i in range(2, 240) if ((i)**2 < (0.45)*(X.shape[0]* X.shape[1])) and i <= min(X.shape[0], X.shape[1])]

    LAMBDA = []

    XX = np.zeros((X.shape[0], X.shape[1]))
    XX[X==2] = 1
  
    for box in box_size:
        count = []
        count, edge= np.histogram(np.ravel(conv2d(XX, np.ones((box, box)), mode = 'valid')), bins = [i for i in range(0, (box**2) + 2)])#(box**2) + 1)
        q = count/(XX.shape[0] - box + 1)**2
        s = np.array([i for i in range(0,box**2 + 1)])
        lam_bda =  sum((s**2)*q)/(sum(q*s))**2


        LAMBDA.append(lam_bda)
        
    return np.nanmean(LAMBDA)

In [9]:
def lac4(x):
    bin_ = np.zeros((x.shape[0], x.shape[1]))
    bin_[x!=0] = 1
    bin_ = bin_.astype(np.uint8)

    contours = cv2.findContours(image = bin_, mode = cv2.RETR_TREE, method = cv2.CHAIN_APPROX_SIMPLE)[0]
    contours = sorted(contours, key = cv2.contourArea, reverse = True)
    
    if len(contours) == 0:
        return 0
    
    c_0 = contours[0]               #Get the 4 points of the bounding rectangle
    xx, yy, w, h = cv2.boundingRect(c_0)
    X  =  x[yy:yy+h, xx:xx+w]
    
    box_size =  [i for i in range(2, 240) if ((i)**2 < (0.45)*(X.shape[0]* X.shape[1])) and i <= min(X.shape[0], X.shape[1])]

    LAMBDA = []
    
    XX = np.zeros((X.shape[0], X.shape[1]))
    
    XX[X==4] = 1
  
    for box in box_size:
        count = []
        count, edge= np.histogram(np.ravel(conv2d(XX, np.ones((box, box)), mode = 'valid')), bins = [i for i in range(0, (box**2) + 2)])#(box**2) + 1)
        q = count/(XX.shape[0] - box + 1)**2
        s = np.array([i for i in range(0,box**2 + 1)])
        lam_bda =  sum((s**2)*q)/(sum(q*s))**2

        LAMBDA.append(lam_bda)
    # print(LAMBDA)
        
    return np.mean(LAMBDA)

In [10]:
def frac_lac(X):
    X = X.astype(np.uint8)

    X = np.delete(X, [i for i in range(X.shape[0]) if np.sum(X[i, :, :].ravel()) < 15], 0)
    

    X_n = np.delete(X, [i for i in range(X.shape[0]) if np.sum(X[i, :, :].ravel() == 1) < 12], 0)
    X_et = np.delete(X, [i for i in range(X.shape[0]) if np.sum(X[i, :, :].ravel() == 4) < 6], 0)
    X_ed = np.delete(X, [i for i in range(X.shape[0]) if np.sum(X[i, :, :].ravel() == 2) < 6], 0)
    

    X_n_slices = [X_n[i, :, :] for i in range(X_n.shape[0])]
    X_et_slices = [X_et[i, :, :] for i in range(X_et.shape[0])]
    X_ed_slices = [X_ed[i, :, :] for i in range(X_ed.shape[0])]
    

    nc_stats = np.array([lac1(x) for x in X_n_slices])
    ed_stats = np.array([lac2(x) for x in X_ed_slices])
    et_stats = np.array([lac4(x) for x in X_et_slices])


    n_mean, n_med = np.nanmean(nc_stats), np.nanmedian(nc_stats)
    ed_mean, ed_med = np.nanmean(ed_stats), np.nanmedian(ed_stats)
    et_mean, et_med = np.nanmean(et_stats), np.nanmedian(et_stats)

    # Return statistics as numpy array
    return np.array([n_mean, n_med, ed_mean, ed_med, et_mean, et_med])


In [11]:
gbm_lac = []
t0 = time.time()

with open('gbm_input.txt') as file:    
    for fl in file:
        try:
            gbm_lac.append(frac_lac(nb.load(r'{}'.format(fl.rstrip())).get_fdata()))
        except:
            continue
        
print('Runtime: ', time.time() - t0 , 'seconds')  
x = np.array(gbm_lac)


case_id = []
with open('gbm_input.txt') as file:
    for fl in file:
        case_id.append(fl.split('\\')[5])    #Number is the position of subject code in tha path filename
        
print(case_id[0])    #to check what is being printed
        
        
gbm_lac_df = pd.DataFrame({'case_id':case_id,
                          'ncr_net_meanlac': x[:, 0], 'ncr_net_medlac': x[:, 1],
                         'ed_meanlac': x[:, 2], 'ed_medlac':x[:, 3],
                         'et_meanlac': x[:, 4], 'et_medlac': x[:, 5]})

# gbm_lac.to_csv('gbm_lac.csv')

Runtime:  1720.4136011600494 seconds
TCGA-02-0006


In [12]:
lgg_lac = []
t0 = time.time()

with open('lgg_input.txt') as file:    
    for fl in file:
        try:
            lgg_lac.append(frac_lac(nb.load(r'{}'.format(fl.rstrip())).get_fdata()))
        except:
            continue
        
print('Runtime:', time.time() - t0 , 'seconds')  
lgg_lac  = np.array(lgg_lac)
x = lgg_lac


case_id2 = []

with open('lgg_input.txt') as file:
    for fl in file:
        case_id2.append(fl.split('\\')[5])    #Number is the position of subject code in tha path filename
        
print(case_id2[0])    #to check what is being printed
        
        
lgg_lac_df = pd.DataFrame({'case_id':case_id2,
                          'ncr_net_meanlac': x[:, 0], 'ncr_net_medlac': x[:, 1],
                         'ed_meanlac': x[:, 2], 'ed_medlac':x[:, 3],
                         'et_meanlac': x[:, 4], 'et_medlac': x[:, 5]})

# lgg_lac.to_csv('lgg_lac.csv')

Runtime: 1022.6195237636566 seconds
TCGA-CS-4942


In [13]:
df = pd.concat([gbm_lac_df, lgg_lac_df])

df.rename(columns={'case_id':'case_id',
                          'ncr_net_meanlac': 'ncr_net_meanlac_sag', 'ncr_net_medlac':'ncr_net_medlac_sag',
                         'ed_meanlac': 'ed_meanlac_sag', 'ed_medlac':'ed_medlac_sag',
                         'et_meanlac': 'et_meanlac_sag', 'et_medlac': 'et_medlac_sag'}, inplace=True)

df.to_csv('lac_sag_merged.csv', index=False)